## DIA 049: Implementación de un Sistema de Registro de Actividad de Usuarios

En este día, se implementa un sistema de registro de actividad de usuarios para auditar interacciones clave dentro de la API. Este sistema almacena eventos como inicio de sesión, intentos fallidos, predicciones realizadas y cambios en configuraciones importantes.

Este mecanismo mejora la seguridad, transparencia y trazabilidad, permitiendo a los administradores revisar eventos sospechosos o patrones de uso inusuales.

🖥️ Código Completo (api.py)
python
Copiar
Editar
import os
import json
import logging
from datetime import datetime
from functools import wraps

from flask import Flask, request, jsonify
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address

# Configuración básica
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'your_secret_key')
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'your_jwt_secret_key')
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///app.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Inicialización de extensiones
db = SQLAlchemy(app)
migrate = Migrate(app, db)
jwt = JWTManager(app)
limiter = Limiter(app, key_func=get_remote_address, default_limits=["100 per day", "20 per hour"])

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------
# Modelos
# ---------------------------
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)

class ActivityLog(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)
    action = db.Column(db.String(120), nullable=False)
    details = db.Column(db.Text, nullable=True)
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)

    def to_dict(self):
        return {
            "id": self.id,
            "username": self.username,
            "action": self.action,
            "details": self.details,
            "timestamp": self.timestamp.isoformat()
        }

# ---------------------------
# Función para Registrar Actividad
# ---------------------------
def log_activity(username, action, details=""):
    activity = ActivityLog(username=username, action=action, details=details)
    db.session.add(activity)
    db.session.commit()
    logger.info(f"Actividad registrada: {username} - {action} - {details}")

# ---------------------------
# Endpoints
# ---------------------------
@app.route('/login', methods=['POST'])
def login():
    """
    Iniciar sesión y registrar el evento.
    """
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')  # Aquí iría la verificación real

    if not username or not password:
        log_activity(username, "failed_login", "Intento de inicio de sesión fallido")
        return jsonify({"msg": "Credenciales incorrectas"}), 400

    token = create_access_token(identity=username)
    log_activity(username, "login", "Inicio de sesión exitoso")
    return jsonify(access_token=token), 200

@app.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("50 per hour")
def predict():
    """
    Endpoint de predicción con registro de actividad.
    """
    current_user = get_jwt_identity()
    log_activity(current_user, "predict", "Usuario realizó una predicción")
    return jsonify({"msg": "Predicción registrada"}), 200

@app.route('/admin/activity_logs', methods=['GET'])
@jwt_required()
def get_activity_logs():
    """
    Obtener registros de actividad (solo admins).
    """
    logs = ActivityLog.query.order_by(ActivityLog.timestamp.desc()).all()
    return jsonify([log.to_dict() for log in logs]), 200

# ---------------------------
# Ejecutar la aplicación
# ---------------------------
if __name__ == '__main__':
    app.run(debug=True)
🔍 Explicación de las Principales Implementaciones
🔹 Nuevo Modelo ActivityLog

Almacena registros de actividad con:
username: Nombre del usuario que realizó la acción.
action: Tipo de acción (login, predicción, intento fallido, etc.).
details: Información adicional sobre la acción.
timestamp: Momento en que ocurrió el evento.
🔹 Función log_activity(username, action, details)

Registra automáticamente eventos de usuario en la base de datos.
Cada evento también se registra en el log del sistema (logger.info).
🔹 Integración con Autenticación JWT

Se requiere autenticación JWT para acceder a los endpoints protegidos.
Cada usuario autenticado deja un rastro de actividad.
🔹 Protección con Rate-Limiting (Flask-Limiter)

Previene abusos limitando la cantidad de llamadas a la API.
Máximo 100 solicitudes diarias y 50 predicciones por hora.
🔹 Nuevos Endpoints
✅ /login

Genera un token JWT y registra intentos de inicio de sesión exitosos y fallidos.
✅ /predict
Simula una predicción y registra el uso del servicio.
✅ /admin/activity_logs
Retorna el historial de actividad de los usuarios (para administradores).
